First Part

In [1]:
import nltk
import pandas as pd
from google.colab import drive
import codecs
from bs4 import BeautifulSoup
import os
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
!pip install texttable

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
queries = pd.read_csv("drive/MyDrive/Colab Notebooks/FUM_IR_1402-02_HW#1_Resources/queries.txt", delimiter = "\t",names=["text"])
# relevance = pd.read_csv("drive/MyDrive/Colab Notebooks/FUM_IR_1402-02_HW#1_Resources/relevance.txt", delimiter = "\t",names=["item"])


In [5]:

def get_files_in_folder(folder_path):
    files = []
    for file in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file)):
            files.append(file)
    return sorted(files, key=lambda x: int(x))

In [15]:

folder_path = "drive/MyDrive/Colab Notebooks/FUM_IR_1402-02_HW#1_Resources/cranfieldDocs"
files_in_folder = get_files_in_folder(folder_path)
print(files_in_folder)
df = pd.DataFrame(columns=['docID','Title', 'Text'])




['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0011', '0012', '0013', '0014', '0015', '0016', '0017', '0018', '0019', '0020', '0021', '0022', '0023', '0024', '0025', '0026', '0027', '0028', '0029', '0030', '0031', '0032', '0033', '0034', '0035', '0036', '0037', '0038', '0039', '0040', '0041', '0042', '0043', '0044', '0045', '0046', '0047', '0048', '0049', '0050', '0051', '0052', '0053', '0054', '0055', '0056', '0057', '0058', '0059', '0060', '0061', '0062', '0063', '0064', '0065', '0066', '0067', '0068', '0069', '0070', '0071', '0072', '0073', '0074', '0075', '0076', '0077', '0078', '0079', '0080', '0081', '0082', '0083', '0084', '0085', '0086', '0087', '0088', '0089', '0090', '0091', '0092', '0093', '0094', '0095', '0096', '0097', '0098', '0099', '0100', '0101', '0102', '0103', '0104', '0105', '0106', '0107', '0108', '0109', '0110', '0111', '0112', '0113', '0114', '0115', '0116', '0117', '0118', '0119', '0120', '0121', '0122', '0123', '0124', '0125',

All of the doc (1400)

In [16]:
for f in files_in_folder:
  with codecs.open(folder_path +"/"+ f, 'r', 'utf-8') as file:
        content = file.read()
        soup = BeautifulSoup(content, "html.parser")
        title_text = soup.title.get_text()
        body_text = soup.find('text').get_text() + title_text
        # print(body_text)
        new_row = pd.DataFrame({'docID' : [f],'Title': [title_text], 'Text': [body_text]})
        df = pd.concat([df, new_row], ignore_index=True)
        # print("---------------------")

In [ ]:
df


,docID,Title,Text
0,0001,\nexperimental investigation of the aerodynami...,\n an experimental study of a wing in a prope...
1,0002,\nsimple shear flow past a flat plate in an in...,\nin the study of high-speed viscous flow past...
2,0003,\nthe boundary layer in simple shear flow past...,\nthe boundary-layer equations are presented f...
3,0004,\napproximate solutions of the incompressible ...,\n the two-dimensional steady boundary-layer\...
4,0005,\none-dimensional transient heat conduction in...,\n analytic solutions are presented for the t...
...,...,...,...
1395,1396,\nshear buckling of clamped and simply-support...,\n the paper presents a solution to the buckl...
1396,1397,\ncritical shear stress of an infinitely long ...,\n a theoretical solution is given for the cr...
1397,1398,\nstability of rectangular plates under shear ...,\n the author first discusses the problem of ...
1398,1399,\nbuckling of transverse stiffened plates unde...,\n this paper presents an analysis of bucklin...


In [7]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    return tokens

In [10]:
def delete_stopwords(tokens):
  stop_words = set(stopwords.words('english'))
  tokens_without_stopwords = [token.lower() for token in tokens if token.lower() not in stop_words]
  return tokens_without_stopwords

In [11]:
def stemming(tokens):
  stemmer = PorterStemmer()
  stemmed_tokens = [stemmer.stem(token) for token in tokens]
  return stemmed_tokens

In [12]:
def delete_punctuation(tokens):

  punctuation_marks = ['.', ',', '?', '!', ':', ';', "'", '"', '(', ')', '[', ']', '{', '}', '-', '—', '...', '/', '\\', '&', '*', '+', '-', '%', '$',
                       '#', '@', '=', '<', '>', '|', '~', '^', '`']
  tokens_without_punctuation = [token for token in tokens if token not in punctuation_marks]
    # Remove punctuation marks from tokens while preserving the tokens
  for punk in punctuation_marks:
        tokens_without_punctuation = [token.replace(punk, '').strip() for token in tokens_without_punctuation]

  return tokens_without_punctuation

In [13]:
def check_format(tokens):

  big_tokens = [token for token in tokens if len(token)>2 and not token.isdigit()]
  return big_tokens

In [17]:
df['tokens_text'] = df['Text'].apply(tokenize_text)
df['tokens_text'] = df['tokens_text'].apply(delete_stopwords).apply(stemming).apply(delete_stopwords)
df['tokens_text'] = df['tokens_text'] .apply(delete_punctuation).apply(check_format)

In [18]:
df['tokens_title'] = df['Title'].apply(tokenize_text)
df['tokens_title'] = df['tokens_title'].apply(delete_stopwords).apply(stemming).apply(delete_stopwords)
df['tokens_title'] = df['tokens_title'] .apply(delete_punctuation).apply(check_format)

In [19]:
queries["token"] = queries["text"].apply(tokenize_text)
queries['normalized'] = queries['token'].apply(delete_stopwords).apply(stemming).apply(delete_stopwords)
queries['normalized'] = queries['normalized'] .apply(delete_punctuation).apply(check_format)

Second Part

In [20]:

df['tokens_text_str'] = df['tokens_text'].apply(lambda tokens: ' '.join(tokens))

vectorizer = TfidfVectorizer()

# Fit and transform the tokens_text_str to get TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(df['tokens_text_str'])
print("tfidf_matrix")
print(tfidf_matrix)
# Convert the TF-IDF matrix to a DataFrame for better readability (optional)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out(), index = df['docID'])

# # Display the TF-IDF DataFrame
print("TF-IDF DataFrame:")
print(tfidf_df)

# # Create an index (dictionary) that maps doc_id to its TF-IDF vector
index = {doc_id: tfidf_matrix[i] for i, doc_id in enumerate(df['docID'])}
print(index)

# Example of accessing the TF-IDF vector for a specific document
# doc_id = 1
# tfidf_vector = index[doc_id]
# print(f"\nTF-IDF vector for document {doc_id}:")
# print(tfidf_vector)


tfidf_matrix
  (0, 235)	0.06325945046984517
  (0, 2568)	0.04833142114853785
  (0, 1720)	0.06601865654726762
  (0, 1001)	0.07882843178281293
  (0, 4669)	0.08457047445821363
  (0, 1572)	0.10164345112940779
  (0, 5065)	0.043725874625492496
  (0, 1878)	0.03435264450327261
  (0, 3760)	0.08629735209912533
  (0, 5637)	0.06656273459652051
  (0, 265)	0.08524390348607297
  (0, 1942)	0.05067255751698515
  (0, 1307)	0.29353897950192426
  (0, 4888)	0.14676948975096213
  (0, 4130)	0.08858167585804479
  (0, 2514)	0.06712161929373439
  (0, 1513)	0.08031186597047554
  (0, 662)	0.15509682884816262
  (0, 1308)	0.15509682884816262
  (0, 3854)	0.08012053990166028
  (0, 2424)	0.21994158045321657
  (0, 4885)	0.09256920262096294
  (0, 4477)	0.0603126593853439
  (0, 1681)	0.09631367630758098
  (0, 4932)	0.08297760319934824
  :	:
  (1399, 3564)	0.18852195260805596
  (1399, 562)	0.08580754015495352
  (1399, 1924)	0.05989149201552099
  (1399, 702)	0.21718119663153826
  (1399, 1534)	0.08204128814613736
  (1399, 73

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [49]:
from texttable import Texttable
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_query(number,queries):
    df['tokens_text_str'] = df['tokens_text'].apply(lambda tokens: ' '.join(tokens))

    # Convert the lists of query tokens to space-separated strings
    queries_str = [' '.join(query) for query in queries['normalized']]

    # Initialize TfidfVectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the tokens_text_str to get TF-IDF matrix for documents
    tfidf_matrix_docs = vectorizer.fit_transform(df['tokens_text_str'])

    # Transform the queries using the same vectorizer
    tfidf_matrix_queries = vectorizer.transform(queries_str)

    # Calculate cosine similarity between queries and documents
    cosine_similarities = cosine_similarity(tfidf_matrix_queries, tfidf_matrix_docs)

    # Get top 20 documents for each query based on cosine similarity
    top_docs = {}
    for i, query in enumerate(queries_str):
        similarity_scores = cosine_similarities[i]
        top_indices = similarity_scores.argsort()[-number:][::-1]
        top_doc_ids = df['docID'].iloc[top_indices].tolist()
        top_docs[query] = top_doc_ids

    # Display the top documents for each query
    table = Texttable()
    table.set_cols_align(["l", "l"])
    table.set_cols_valign(["m", "m"])
    table.add_row(["Query", "Top Document IDs"])
    i = 0
    for query, top_doc_ids in top_docs.items():
        doc_ids_str = ", ".join(map(str, top_doc_ids))
        table.add_row([query, doc_ids_str])

        for doc in top_doc_ids:
          print(f"({i} , {doc} )")
        i += 1
    result = table.draw()
    return top_docs , result

In [50]:
top_docs,table = get_query(20,queries)
print(table)

(0 , 0958 )
(0 , 0175 )
(0 , 0407 )
(0 , 0764 )
(0 , 0971 )
(0 , 0064 )
(0 , 1156 )
(0 , 1220 )
(0 , 0974 )
(0 , 0693 )
(0 , 1225 )
(0 , 0039 )
(0 , 0062 )
(0 , 1288 )
(0 , 1319 )
(0 , 0367 )
(0 , 1269 )
(0 , 0117 )
(0 , 0239 )
(0 , 0174 )
(1 , 0559 )
(1 , 0655 )
(1 , 0323 )
(1 , 1319 )
(1 , 0666 )
(1 , 1274 )
(1 , 0554 )
(1 , 0629 )
(1 , 0983 )
(1 , 0324 )
(1 , 1161 )
(1 , 0180 )
(1 , 0369 )
(1 , 0318 )
(1 , 1263 )
(1 , 0523 )
(1 , 0123 )
(1 , 0398 )
(1 , 0211 )
(1 , 0626 )
(2 , 0559 )
(2 , 0323 )
(2 , 0655 )
(2 , 0666 )
(2 , 0629 )
(2 , 0554 )
(2 , 0983 )
(2 , 0180 )
(2 , 0398 )
(2 , 0211 )
(2 , 0324 )
(2 , 0295 )
(2 , 0145 )
(2 , 0433 )
(2 , 0154 )
(2 , 0902 )
(2 , 1263 )
(2 , 0120 )
(2 , 1393 )
(2 , 0044 )
(3 , 1078 )
(3 , 0668 )
(3 , 1222 )
(3 , 0197 )
(3 , 1221 )
(3 , 0053 )
(3 , 0775 )
(3 , 1191 )
(3 , 0009 )
(3 , 0530 )
(3 , 0992 )
(3 , 1082 )
(3 , 1273 )
(3 , 0579 )
(3 , 0208 )
(3 , 0139 )
(3 , 0267 )
(3 , 0342 )
(3 , 1236 )
(3 , 0504 )
(4 , 0004 )
(4 , 0738 )
(4 , 0292 )
(4 ,

In [27]:
reldoc = pd.read_csv("drive/MyDrive/Colab Notebooks/FUM_IR_1402-02_HW#1_Resources/relevance.txt", header=None, delimiter=' ', names=['query_id', 'doc_id'])

print(reldoc)

     query_id  doc_id
0           1     156
1           2     666
2           2     667
3           2    1258
4           2    1394
..        ...     ...
126        10     924
127        10    1062
128        10    1074
129        10    1075
130        10    1213

[131 rows x 2 columns]


In [ ]:
reldoc['query_id']

0       1
1       2
2       2
3       2
4       2
       ..
126    10
127    10
128    10
129    10
130    10
Name: query_id, Length: 131, dtype: int64

In [28]:
answers  = []
row= []
last_qi = 0
j = 0
for i in reldoc['query_id']:
  if(last_qi < i-1):
    last_qi = i-1
    answers.append(row)
    row= []
  # print(last_qi)
  row.append(reldoc.loc[j,'doc_id'])
  # answers[last_qi].append(reldoc.loc[j,'doc_id'])
  j += 1
print(last_qi)
answers.append(row)
print(answers)

9
[[156], [666, 667, 1258, 1394, 668, 670, 1204, 1391, 1395, 1300, 37, 559, 630, 1107, 1213], [24, 101, 666, 667, 93, 1258, 1393, 559, 630, 662, 1104, 1107, 1204, 1213, 1300], [1391, 666, 667, 1258, 1078, 1080, 1081, 1394, 1395, 1214, 1198, 1204, 1300, 559, 630, 662, 1107, 1213], [1383, 1385, 155, 241, 1382, 1370, 1386, 111, 1384, 150, 292, 458, 479, 977, 376, 459, 1365, 62, 1366], [155, 1383, 1385, 1382, 62, 292, 241, 1370, 1384, 458, 459, 461, 1386, 1365, 1366, 111, 150, 479], [400, 419, 1387, 412, 1392, 1398, 1397, 1400, 1399], [400, 1387, 1392, 1398], [656, 1313, 1317, 1316, 1318, 1319, 1157, 1274], [1379, 1305, 1304, 40, 293, 1309, 161, 421, 1377, 1378, 1381, 225, 1380, 448, 449, 1124, 1280, 433, 923, 924, 1062, 1074, 1075, 1213]]


In [30]:
def calculate_precision(answers,results):
  true_positiv = 0
  for doc in results:
    if doc in answers:
      true_positiv += 1
  return true_positiv/len(answers)


In [31]:
def calculate_recall(answers,results):
  true_positiv = 0
  for doc in results:
    if doc in answers:
      true_positiv += 1
  return true_positiv/len(results)

In [34]:
i = 0
sum_recall = 0
sum_precision = 0
for query, top_doc_ids in top_docs.items():
    print(f"Query: {query}")
    result =  [int(docid) for docid  in top_doc_ids]
    prec = calculate_precision(answers[i],result)
    sum_precision += prec
    recall = calculate_recall(answers[i],result)
    sum_recall += recall
    print(f"precision is {prec}")
    print(f"recall is {recall}")
    print("------------------------")
    i += 1

print(f"mean of recall is {sum_recall/len(answers)}")
print(f"mean of precision is {sum_precision/len(answers)}")

Query: investig made wave system creat static pressur distribut liquid surfac
precision is 0.0
recall is 0.0
------------------------
Query: anyon investig effect shock gener vortic heat transfer blunt bodi
precision is 0.13333333333333333
recall is 0.1
------------------------
Query: heat transfer blunt bodi absenc vortic
precision is 0.2
recall is 0.15
------------------------
Query: gener effect flow field reynold number small
precision is 0.05555555555555555
recall is 0.05
------------------------
Query: find calcul procedur applic incompress laminar boundari layer flow problem good accuraci reason comput time
precision is 0.21052631578947367
recall is 0.2
------------------------
Query: paper applic problem calcul procedur laminar incompress flow arbitrari pressur gradient
precision is 0.2222222222222222
recall is 0.2
------------------------
Query: anyon investig shear buckl stiffen plate
precision is 0.8888888888888888
recall is 0.4
------------------------
Query: paper shear bu

In [51]:
top_docs50,table = get_query(50,queries)
print(table)
i = 0
sum_recall = 0
sum_precision = 0
for query, top_doc_ids in top_docs50.items():
    print(f"Query: {query}")
    result =  [int(docid) for docid  in top_doc_ids]
    prec = calculate_precision(answers[i],result)
    sum_precision += prec
    recall = calculate_recall(answers[i],result)
    sum_recall += recall
    print(f"precision is {prec}")
    print(f"recall is {recall}")
    print("------------------------")
    i += 1

print(f"mean of recall is {sum_recall/len(answers)}")
print(f"mean of precision is {sum_precision/len(answers)}")

(0 , 0958 )
(0 , 0175 )
(0 , 0407 )
(0 , 0764 )
(0 , 0971 )
(0 , 0064 )
(0 , 1156 )
(0 , 1220 )
(0 , 0974 )
(0 , 0693 )
(0 , 1225 )
(0 , 0039 )
(0 , 0062 )
(0 , 1288 )
(0 , 1319 )
(0 , 0367 )
(0 , 1269 )
(0 , 0117 )
(0 , 0239 )
(0 , 0174 )
(0 , 1032 )
(0 , 0411 )
(0 , 0065 )
(0 , 1364 )
(0 , 0904 )
(0 , 0808 )
(0 , 0636 )
(0 , 1389 )
(0 , 0512 )
(0 , 0612 )
(0 , 0345 )
(0 , 0914 )
(0 , 0692 )
(0 , 0993 )
(0 , 1166 )
(0 , 1231 )
(0 , 0133 )
(0 , 0514 )
(0 , 0335 )
(0 , 0665 )
(0 , 1274 )
(0 , 1352 )
(0 , 1382 )
(0 , 0147 )
(0 , 1000 )
(0 , 0968 )
(0 , 1206 )
(0 , 0905 )
(0 , 0072 )
(0 , 0132 )
(1 , 0559 )
(1 , 0655 )
(1 , 0323 )
(1 , 1319 )
(1 , 0666 )
(1 , 1274 )
(1 , 0554 )
(1 , 0629 )
(1 , 0983 )
(1 , 0324 )
(1 , 1161 )
(1 , 0180 )
(1 , 0369 )
(1 , 0318 )
(1 , 1263 )
(1 , 0523 )
(1 , 0123 )
(1 , 0398 )
(1 , 0211 )
(1 , 0626 )
(1 , 0044 )
(1 , 0145 )
(1 , 1157 )
(1 , 0120 )
(1 , 0572 )
(1 , 0433 )
(1 , 1364 )
(1 , 0190 )
(1 , 0372 )
(1 , 0623 )
(1 , 0438 )
(1 , 0902 )
(1 , 0037 )
(1 ,

In [52]:
top_docs100,table = get_query(100,queries)
print(table)
i = 0
sum_recall = 0
sum_precision = 0
for query, top_doc_ids in top_docs100.items():
    print(f"Query: {query}")
    result =  [int(docid) for docid  in top_doc_ids]
    prec = calculate_precision(answers[i],result)
    sum_precision += prec
    recall = calculate_recall(answers[i],result)
    sum_recall += recall
    print(f"precision is {prec}")
    print(f"recall is {recall}")
    print("------------------------")
    i += 1

print(f"mean of recall is {sum_recall/len(answers)}")
print(f"mean of precision is {sum_precision/len(answers)}")

(0 , 0958 )
(0 , 0175 )
(0 , 0407 )
(0 , 0764 )
(0 , 0971 )
(0 , 0064 )
(0 , 1156 )
(0 , 1220 )
(0 , 0974 )
(0 , 0693 )
(0 , 1225 )
(0 , 0039 )
(0 , 0062 )
(0 , 1288 )
(0 , 1319 )
(0 , 0367 )
(0 , 1269 )
(0 , 0117 )
(0 , 0239 )
(0 , 0174 )
(0 , 1032 )
(0 , 0411 )
(0 , 0065 )
(0 , 1364 )
(0 , 0904 )
(0 , 0808 )
(0 , 0636 )
(0 , 1389 )
(0 , 0512 )
(0 , 0612 )
(0 , 0345 )
(0 , 0914 )
(0 , 0692 )
(0 , 0993 )
(0 , 1166 )
(0 , 1231 )
(0 , 0133 )
(0 , 0514 )
(0 , 0335 )
(0 , 0665 )
(0 , 1274 )
(0 , 1352 )
(0 , 1382 )
(0 , 0147 )
(0 , 1000 )
(0 , 0968 )
(0 , 1206 )
(0 , 0905 )
(0 , 0072 )
(0 , 0132 )
(0 , 1378 )
(0 , 1311 )
(0 , 0569 )
(0 , 1004 )
(0 , 0222 )
(0 , 0268 )
(0 , 0694 )
(0 , 0492 )
(0 , 0961 )
(0 , 0655 )
(0 , 1356 )
(0 , 0556 )
(0 , 1102 )
(0 , 0256 )
(0 , 0037 )
(0 , 1001 )
(0 , 1151 )
(0 , 0423 )
(0 , 0662 )
(0 , 0330 )
(0 , 1321 )
(0 , 0049 )
(0 , 1200 )
(0 , 0296 )
(0 , 1039 )
(0 , 0713 )
(0 , 0626 )
(0 , 1292 )
(0 , 0794 )
(0 , 0529 )
(0 , 0824 )
(0 , 0056 )
(0 , 1145 )
(0 ,

# **Our Queries**

In [55]:
our_queries = pd.read_csv("/content/OurQueries.txt", delimiter = "\t",names=["text"])
our_queries["token"] = our_queries["text"].apply(tokenize_text)
our_queries['normalized'] = our_queries['token'].apply(delete_stopwords).apply(stemming).apply(delete_stopwords)
our_queries['normalized'] = our_queries['normalized'] .apply(delete_punctuation).apply(check_format)

In [56]:
our_reldoc = pd.read_csv("/content/OurAnswers.txt", header=None, delimiter=' ', names=['query_id', 'doc_id'])

print(our_reldoc)
our_answers  = []
row= []
last_qi = 0
j = 0
for i in our_reldoc['query_id']:
  if(last_qi < i-1):
    last_qi = i-1
    our_answers.append(row)
    row= []
  # print(last_qi)
  row.append(our_reldoc.loc[j,'doc_id'])
  # answers[last_qi].append(reldoc.loc[j,'doc_id'])
  j += 1
print(last_qi)
our_answers.append(row)
print(our_answers)

    query_id  doc_id
0          1      19
1          1       4
2          1       1
3          1       6
4          2     962
5          2    1002
6          2    1340
7          3    1392
8          3     515
9          3     474
10         3      89
11         4      41
12         4     200
13         4     432
14         4     524
15         5     367
16         5       7
17         5     780
18         5     911
19         5    1045
4
[[19, 4, 1, 6], [962, 1002, 1340], [1392, 515, 474, 89], [41, 200, 432, 524], [367, 7, 780, 911, 1045]]


In [59]:
top_docs,table = get_query(10,our_queries)
print(table)
i = 0
sum_recall = 0
sum_precision = 0
for query, top_doc_ids in top_docs.items():
    print(f"Query: {query}")
    result =  [int(docid) for docid  in top_doc_ids]
    prec = calculate_precision(our_answers[i],result)
    sum_precision += prec
    recall = calculate_recall(our_answers[i],result)
    sum_recall += recall
    print(f"precision is {prec}")
    print(f"recall is {recall}")
    print("------------------------")
    i += 1

print(f"mean of recall is {sum_recall/len(our_answers)}")
print(f"mean of precision is {sum_precision/len(our_answers)}")

(0 , 1154 )
(0 , 0959 )
(0 , 1366 )
(0 , 0406 )
(0 , 0899 )
(0 , 0393 )
(0 , 0004 )
(0 , 0045 )
(0 , 1281 )
(0 , 1295 )
(1 , 1002 )
(1 , 1340 )
(1 , 0959 )
(1 , 0145 )
(1 , 0294 )
(1 , 0554 )
(1 , 0378 )
(1 , 1263 )
(1 , 1366 )
(1 , 0559 )
(2 , 0658 )
(2 , 0857 )
(2 , 1115 )
(2 , 0856 )
(2 , 0350 )
(2 , 0515 )
(2 , 1008 )
(2 , 0207 )
(2 , 0627 )
(2 , 0240 )
(3 , 0287 )
(3 , 0295 )
(3 , 0790 )
(3 , 0060 )
(3 , 0288 )
(3 , 0437 )
(3 , 0208 )
(3 , 0123 )
(3 , 0041 )
(3 , 1115 )
(4 , 0532 )
(4 , 0367 )
(4 , 0284 )
(4 , 1045 )
(4 , 0051 )
(4 , 1163 )
(4 , 1264 )
(4 , 0839 )
(4 , 1169 )
(4 , 0253 )
+---------------------------------------+--------------------------------------+
| Query                                 | Top Document IDs                     |
+---------------------------------------+--------------------------------------+
| recent studi aerodynam heat flow      | 1154, 0959, 1366, 0406, 0899, 0393,  |
| boundari layer equat                  | 0004, 0045, 1281, 1295            

In [57]:
top_docs,table = get_query(20,our_queries)
print(table)
i = 0
sum_recall = 0
sum_precision = 0
for query, top_doc_ids in top_docs.items():
    print(f"Query: {query}")
    result =  [int(docid) for docid  in top_doc_ids]
    prec = calculate_precision(our_answers[i],result)
    sum_precision += prec
    recall = calculate_recall(our_answers[i],result)
    sum_recall += recall
    print(f"precision is {prec}")
    print(f"recall is {recall}")
    print("------------------------")
    i += 1

print(f"mean of recall is {sum_recall/len(our_answers)}")
print(f"mean of precision is {sum_precision/len(our_answers)}")

(0 , 1154 )
(0 , 0959 )
(0 , 1366 )
(0 , 0406 )
(0 , 0899 )
(0 , 0393 )
(0 , 0004 )
(0 , 0045 )
(0 , 1281 )
(0 , 1295 )
(0 , 0145 )
(0 , 0024 )
(0 , 1263 )
(0 , 0003 )
(0 , 0459 )
(0 , 0256 )
(0 , 1383 )
(0 , 0629 )
(0 , 0655 )
(0 , 0005 )
(1 , 1002 )
(1 , 1340 )
(1 , 0959 )
(1 , 0145 )
(1 , 0294 )
(1 , 0554 )
(1 , 0378 )
(1 , 1263 )
(1 , 1366 )
(1 , 0559 )
(1 , 0655 )
(1 , 0748 )
(1 , 0037 )
(1 , 0666 )
(1 , 0638 )
(1 , 1301 )
(1 , 1185 )
(1 , 0192 )
(1 , 0708 )
(1 , 1192 )
(2 , 0658 )
(2 , 0857 )
(2 , 1115 )
(2 , 0856 )
(2 , 0350 )
(2 , 0515 )
(2 , 1008 )
(2 , 0207 )
(2 , 0627 )
(2 , 0240 )
(2 , 1392 )
(2 , 1113 )
(2 , 0859 )
(2 , 0241 )
(2 , 0864 )
(2 , 0894 )
(2 , 0766 )
(2 , 0131 )
(2 , 0544 )
(2 , 0243 )
(3 , 0287 )
(3 , 0295 )
(3 , 0790 )
(3 , 0060 )
(3 , 0288 )
(3 , 0437 )
(3 , 0208 )
(3 , 0123 )
(3 , 0041 )
(3 , 1115 )
(3 , 1161 )
(3 , 0464 )
(3 , 1073 )
(3 , 1264 )
(3 , 0522 )
(3 , 0302 )
(3 , 0601 )
(3 , 0515 )
(3 , 0294 )
(3 , 0524 )
(4 , 0532 )
(4 , 0367 )
(4 , 0284 )
(4 ,

In [58]:
top_docs,table = get_query(50,our_queries)
print(table)
i = 0
sum_recall = 0
sum_precision = 0
for query, top_doc_ids in top_docs.items():
    print(f"Query: {query}")
    result =  [int(docid) for docid  in top_doc_ids]
    prec = calculate_precision(our_answers[i],result)
    sum_precision += prec
    recall = calculate_recall(our_answers[i],result)
    sum_recall += recall
    print(f"precision is {prec}")
    print(f"recall is {recall}")
    print("------------------------")
    i += 1

print(f"mean of recall is {sum_recall/len(our_answers)}")
print(f"mean of precision is {sum_precision/len(our_answers)}")

(0 , 1154 )
(0 , 0959 )
(0 , 1366 )
(0 , 0406 )
(0 , 0899 )
(0 , 0393 )
(0 , 0004 )
(0 , 0045 )
(0 , 1281 )
(0 , 1295 )
(0 , 0145 )
(0 , 0024 )
(0 , 1263 )
(0 , 0003 )
(0 , 0459 )
(0 , 0256 )
(0 , 1383 )
(0 , 0629 )
(0 , 0655 )
(0 , 0005 )
(0 , 0335 )
(0 , 0376 )
(0 , 0378 )
(0 , 0872 )
(0 , 0135 )
(0 , 0337 )
(0 , 1192 )
(0 , 1301 )
(0 , 1268 )
(0 , 0260 )
(0 , 1384 )
(0 , 1185 )
(0 , 1149 )
(0 , 0383 )
(0 , 0527 )
(0 , 0606 )
(0 , 0192 )
(0 , 0303 )
(0 , 1382 )
(0 , 0339 )
(0 , 0458 )
(0 , 0978 )
(0 , 0105 )
(0 , 0507 )
(0 , 0562 )
(0 , 1182 )
(0 , 0255 )
(0 , 1073 )
(0 , 0784 )
(0 , 0348 )
(1 , 1002 )
(1 , 1340 )
(1 , 0959 )
(1 , 0145 )
(1 , 0294 )
(1 , 0554 )
(1 , 0378 )
(1 , 1263 )
(1 , 1366 )
(1 , 0559 )
(1 , 0655 )
(1 , 0748 )
(1 , 0037 )
(1 , 0666 )
(1 , 0638 )
(1 , 1301 )
(1 , 1185 )
(1 , 0192 )
(1 , 0708 )
(1 , 1192 )
(1 , 0544 )
(1 , 0690 )
(1 , 0860 )
(1 , 0021 )
(1 , 0102 )
(1 , 1163 )
(1 , 0360 )
(1 , 0055 )
(1 , 1076 )
(1 , 0798 )
(1 , 0406 )
(1 , 0004 )
(1 , 1239 )
(1 ,